In [2]:
import gym
from gym import wrappers
import numpy as np
import random

In [11]:
for i in gym.envs.registry:
    #print(i)

SyntaxError: incomplete input (159489049.py, line 2)

In [5]:
class Q_Net:
    def __init__(self, input_size, hidden_size, output_size):
        self.learning_rate = 0.995
        self.epsilon = 0.45 
        self.decay_factor = 0.95

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights
        self.W1 = np.random.randn(hidden_size, input_size) * np.sqrt(1/input_size)
        self.b1 = np.zeros((hidden_size, ))
        self.W2 = np.random.randn(output_size, hidden_size) * np.sqrt(1/output_size)
        self.b2 = np.zeros((output_size, ))
    
    def forward(self, state):
        self.hidden_in = np.dot(self.W1, state) + self.b1
        self.hidden_out = self.relu(self.hidden_in)
        self.output_in = np.dot(self.W2, self.hidden_out) + self.b2        
        self.output_out = self.sigmoid(self.output_in)
        return self.output_out
    
    def backprop(self, observation, Error):
        # Backpropagation
        reluOutput = self.sigmoid_backward(self.output_in)
        #print("Shape of ReluOutput: ", reluOutput.shape)

        delta2 = reluOutput * Error 
        #print("Shape of delta 2:", delta2.shape)

        dw2 = np.outer(delta2, self.hidden_out)
        #print("Shape of dw2:", dw2.shape)

        db2 = delta2
        #print("shape of db2:", db2.shape)

        # Backpropagation: hidden layer -> input layer
        reluInput = self.relu_backward(self.hidden_in)
        #print("Shape of ReluInput: ", reluInput.shape)

        delta1 = reluInput * np.dot(self.W2.T, delta2)
        #print("Shape of delta 1:", delta1.shape)

        dw1 = np.outer(delta1, observation)
        #print("Shape of dw1:", dw1.shape)

        db1 = delta1
        #print("Shape of db1:", db1.shape)
       
        # Update Rules
        self.W1 += self.learning_rate * dw1
        self.W2 += self.learning_rate * dw2
        self.b1 += self.learning_rate * db1
        self.b2 += self.learning_rate * db2  
        
    def ep_greedy(self, Q_values):
        if random.random() < self.epsilon:
            return random.randint(0, 1)
        else: 
            return np.argmax(Q_values)
        
    def decay(self):
        self.epsilon *= self.decay_factor

    def softmax(self, Z):
        e_Z = np.exp(Z - np.max(Z, axis=0))
        return e_Z / np.sum(e_Z, axis=0)
    
    def sigmoid(self, Z):
        return 1/(1+np.exp(-Z))

    def relu(self, Z):
        return np.maximum(0,Z)

    def sigmoid_backward(self, layer):
        sig = self.sigmoid(layer)
        return sig * (1 - sig)

    def relu_backward(self, layer):
        return np.where(layer > 0, 1, 0)


In [7]:
gamma = 0.85  

env = gym.make('CartPole-v1', render_mode="human")
state = env.reset()

s_size = env.observation_space.shape[0]
a_size = env.action_space.n
hidden_size = 16

net = Q_Net(s_size, hidden_size, a_size)

reward_avg = 0

for i in range(10000):
    done = False
    state = env.reset()[0]

    while not done:
        
        net.decay()

        # Forward prop
        Q_values = net.forward(state)
        
        # Policy Decision
        Q_current = np.max(Q_values)
        action = net.ep_greedy(Q_values)
        
        # Next step
        observation_next, reward, done, info, _ = env.step(action)
        reward_avg += reward

        if not done: 
            Error = reward - Q_current
            net.backprop(state, Error)
        else:
            Q_values_next = net.forward(observation_next)
            Q_max = np.max(Q_values_next)
            Q_target = reward + gamma * Q_max 
            Error = Q_target - Q_current 
            net.backprop(state, Error)
            if i % 100 == 0:
                print(reward_avg/100)
                reward_avg = 0
        
        state = observation_next
env.close()

0.17
41.8
37.36
40.64
38.23
39.6
38.97
37.48
41.58
44.46
44.6
39.97
38.78
38.05
44.76
41.15
45.85
42.4
46.01
42.92
44.11
44.37
42.69
43.1
40.38
44.69
48.02
42.85
41.59
42.96
47.69
46.03
47.82
42.46
44.4
45.99
42.5
42.1
47.1
45.96
42.79
40.95
44.31
42.29
47.36
48.83
46.16
44.39
46.87
43.74
49.16
45.62
44.23
39.26
42.19
46.12
40.02
48.6
50.18
47.16
41.69
47.59
45.59
45.96
54.11
50.91
47.32
41.24
48.12
45.64
49.93
48.91
45.76
43.26
45.8
47.61
49.29
49.05
49.84
42.73
43.44


KeyboardInterrupt: 

: 